In [1]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [2]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [7]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "Arabic"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# source path
source_path = "/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Deployment2/Result/Deploy2 Result Manuel/Turkish"

# parameter
file = "Basics Cover the Most Master File"  # file without extention
sheets = "85 Sentence"  # 2 gram target, 2 gram hybrid
file_ext = "85 Sentence"
first_column = 2  # start column location number
end_column = 3

# native word select for Part 2 
word_start = 0  # 0  # native word start index
word_end = 200 # 28  # native word end index

In [8]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
987924,karneleme,5
987925,karnaya,5
987926,dörtlümüzün,5
987927,karnavalınız,5


In [9]:
df_word_select = df_word_all.iloc[word_start:word_end,]
df_word_select.reset_index(inplace=True)
df_word_select

,index,word,frequency
0,0,bir,18835735
1,1,bu,11062659
2,2,ne,8025880
3,3,ve,7766036
4,4,için,5484109
...,...,...,...
195,195,saat,399989
196,196,onunla,399330
197,197,yapıyorsun,398274
198,198,neler,397377


In [4]:
df_file = pd.read_excel(f"{source_path}/{file}.xlsx", sheet_name=f"{sheets}")
df_file

,search_string,start_time,end_time,sentence,video_id,video_url,word_index_list
0,kendi şimdi sen mi ne oluyor biliyor musun bu ...,446,451,artık hamill olması lazım altı kendi şimdi sen...,Yuu5VhJBIUg,https://www.youtube.com/watch?v=Yuu5VhJBIUg&t=...,"[133, 43, 19, 5, 2, 139, 109, 53, 1, 43, 168, ..."
1,gerçekten uzun ve bu da benim için çok önemli ...,623,627,bu tabletin şarj ömrü batarya ömrü gerçekten u...,r_9ngjGh2Ks,https://www.youtube.com/watch?v=r_9ngjGh2Ks&t=...,"[70, 172, 3, 1, 15, 31, 4, 9, 156, 68, 67]"
2,her zaman öyle değil herkes için de her zaman ...,559,562,her zaman öyle değil herkes için de her zaman ...,cPoQ27hCNOU,https://www.youtube.com/watch?v=cPoQ27hCNOU&t=...,"[30, 35, 46, 14, 173, 4, 8, 30, 35, 46, 14]"
3,çok iyi değil mi tam böyle gerçek gibi gerçek ...,3,6,çok iyi değil mi tam böyle gerçek gibi gerçek ...,OmT_BtZh8qU,https://www.youtube.com/watch?v=OmT_BtZh8qU&t=3s,"[9, 28, 14, 5, 112, 56, 183, 27, 183, 27, 60]"
4,nasıl bir şekilde olduğunu ve neden o şekilde ...,14,17,şimdi apexin nasıl bir şekilde olduğunu ve ned...,owj1QZoYqFs,https://www.youtube.com/watch?v=owj1QZoYqFs&t=14s,"[39, 0, 181, 40, 3, 34, 6, 181, 40]"
...,...,...,...,...,...,...,...
80,en fazla iki saat,318,320,bu evde en fazla iki saat geçiririm,bt6fLjCmyvM,https://www.youtube.com/watch?v=bt6fLjCmyvM&t=...,"[44, 96, 110, 195]"
81,ol başka biri ol,281,283,mark zuckerberg ol başka biri ol,30Sd_IpkgK4,https://www.youtube.com/watch?v=30Sd_IpkgK4&t=...,"[118, 71, 78, 118]"
82,şu adam şu adam,214,216,şu adam şu adam,mHkBxBLAZww,https://www.youtube.com/watch?v=mHkBxBLAZww&t=...,"[48, 81, 48, 81]"
83,tamam peki pekala pekala tamam,1408,1410,NaN,d9tZZGbLQMU,https://www.youtube.com/watch?v=d9tZZGbLQMU&t=...,NaN


In [11]:
def word_count_result(df,column_list): # df is dataframe, column_list is list value
    '''
    word_count_bool(df, column_list): df columns word count for word frequency\n
    df is dataframe, column_list is list value\n
    word_count_bool(df, ["word","twogram"]):
    '''
    list_all = []
    for i in df.loc[:,[x for x in column_list]].columns:
        var_list = df[f"{i}"].dropna().tolist()
        for j in var_list:
            list_all.append(j)
    text = " ".join(list_all)
    word_list = re.findall(r"\w+",text, re.UNICODE)
    df_word_list = pd.DataFrame(word_list, columns=["word"])
    #df_word_list.rename(columns={0:"word"}, inplace=True)
    df_word_count = pd.DataFrame(df_word_list.value_counts())
    df_word_count.reset_index(inplace=True)
    df_word_count.rename(columns={0:"word_count"}, inplace=True)
    
    return  df_word_count

In [13]:
df_file_count = word_count_result(df_file, df_file.columns[0:1])
df_file_count.sort_values(by="word_count", ascending=False)
df_file_count

,word,word_count
0,çok,13
1,bir,13
2,güzel,8
3,bakalım,8
4,değil,8
...,...,...
194,istiyorsun,1
195,ister,1
196,hiç,1
197,hey,1
